In [ ]:
import sys
sys.path.insert(0,'../')

import flammkuchen as fl
import tensorflow as tf
from tensorflow.keras.models import load_model
import librosa
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import copy
from nutil.plot import paperStyle
from sklearn.utils.class_weight import compute_class_weight

from neural_networks.src.dataloader import DataLoader
from utils.params import Params
from analysis.src.conf_matrices import generate_confusion_matrix
from neural_networks.src.lr_scheduler import exp_scheduler

%load_ext autoreload
%autoreload 2

In [ ]:
PATH_DATA = r"../../Data/New_Data/"
PATH_GA_MODEL_NSA = r"../neural_networks/GeneticAlgorithm/Results_GA/GA_20211115-105541_objective2/best_models_per_generation/gen17_Fitness2.4656.hdf5"

PATH_PARAMS = r"../neural_networks/params.json"

In [ ]:
params = Params(PATH_PARAMS)

In [ ]:
params.signal_type = 'NSA'
data_loader = DataLoader(params=params, nb_classes=4)
(X_train_new_dataset, Y_train_new_dataset), (X_test_new_dataset, Y_test_new_dataset) = data_loader.get_mcgill_new_data(PATH_DATA, without_no_event=True)

In [ ]:
model_ga = load_model(PATH_GA_MODEL_NSA, custom_objects={'leaky_relu': tf.nn.leaky_relu, 'relu6': tf.nn.relu6})

In [ ]:
model_ga.evaluate(X_test_new_dataset[..., None], Y_test_new_dataset)

# Train the model again

In [ ]:
(X_train_old, Y_train_old), (X_val_old, Y_val_old) = data_loader.get_train_val_data()
X_test_old, Y_test_old = data_loader.get_test_data()

In [ ]:
def fine_tune_model(X, Y):
    model_ga = load_model(PATH_GA_MODEL_NSA, custom_objects={'leaky_relu': tf.nn.leaky_relu, 'relu6': tf.nn.relu6})
    fine_tuned_model = model_ga
    fine_tuned_model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=["accuracy"])
    
    test_accs_old = []
    test_accs_new = []
    
    class_weight = None
    
    n_epochs = 100
    for epoch in range(n_epochs):
        if epoch == 70:
            # after epoch 70 the learning rate is reduced
            fine_tuned_model.compile(optimizer=tf.keras.optimizers.Adam(0.00001), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=["accuracy"])
        history = fine_tuned_model.fit(X_[..., None], Y_, 
                                     shuffle=True, 
                                     epochs=1,
                                     batch_size=256,
                                     class_weight=class_weight,
                                     verbose=False)
        
        # calculate test accuracies after each epoch
        test_accs_old.append(fine_tuned_model.evaluate(X_test_old[..., None], Y_test_old)[1])
        test_accs_new.append(fine_tuned_model.evaluate(X_test_new_dataset[..., None], Y_test_new_dataset)[1])
        
    return test_accs_old, test_accs_new

def get_class_weights(Y_train):
    y_train = []
    for y in Y_train:
        y_train.append(int(np.where(y == 1)[0]) + 1)
    y_train.append(4) # append one 4 in case there is no "no event" in the list
    y_train = np.array(y_train)
    class_weights = compute_class_weight(class_weight='balanced', classes=[1, 2, 3, 4], y=y_train)
    class_weights = {0: class_weights[0],
                     1: class_weights[1],
                     2: class_weights[2],
                     3: class_weights[3]}
    print(f"Using class weights: {class_weights}")
    return class_weights

## Fine tuning with old and new dataset combined

In [ ]:
X_ = np.array([*X_train_old, *X_train_new_dataset])
Y_ = np.array([*Y_train_old, *Y_train_new_dataset])

test_accs_old_1, test_accs_new_1 = fine_tune_model(X_, Y_)

plt.plot(test_accs_old_1, label="old test accs")
plt.plot(test_accs_new_1, label="new test accs")
plt.legend()
plt.show()

## Fine tuning only with new dataset

In [ ]:
X_ = np.array(X_train_new_dataset)
Y_ = np.array(Y_train_new_dataset)

test_accs_old_2, test_accs_new_2 = fine_tune_model(X_, Y_)

plt.plot(test_accs_old_2, label="old test accs")
plt.plot(test_accs_new_2, label="new test accs")
plt.legend()
plt.show()

# Save predictions using 'flammkuchen'

In [ ]:
d = dict(test_accs_old_1=test_accs_old_1, test_accs_new_1=test_accs_new_1, test_accs_old_2=test_accs_old_2, test_accs_new_2=test_accs_new_2)
fl.save("fine_tuning_new_dataset.vfp", d)